<a href="https://colab.research.google.com/github/shlederman/amazon_tracking/blob/main/amazon_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

!pip install bs4 -q
!pip install lxml -q
!pip install requests -q

from datetime import date
from bs4 import BeautifulSoup 
import requests 

drivePath = "/content/drive/MyDrive/Colab Notebooks/Data Files/Amazon/"

today=date.today()



def main(URL):

  
#File to which output is written.  New records are appended to end of file.  
#The today date above is incldued with the row to be able to separate the different entries.
#Date can eventually be used to map prices changes over time.
  outFile = open(drivePath+"amazon tracking.csv", "a") 
  
# Needed to prevent Amazon blocking automated requests to the website
  HEADERS = {
    'Host': 'www.amazon.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:70.0) Gecko/20100101 Firefox/70.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'TE': 'Trailers'
  }
  

  
  #Requests library is used to send HTTP requests and retreive HTML/XML pages from a website
  #BeautifulSoup library is used to obtain the data from HTML/XML files.  Using parsers, the different 
  #components of the site can be identified and captured.
  webpage = requests.get(URL, headers=HEADERS) 
  soup = BeautifulSoup(webpage.content, "lxml")

 
  
  try: 
    title = soup.find("span",  attrs={"id": 'productTitle'}) 
    title_value = title.string 
    title_string = title_value .strip().replace(',', '') 
            
  except AttributeError:
    title_string = "NA"
  
  print("Product Title = ", title_string) 

  # saving the title in the file 
 # outFile.write(f"{title_string},") 


  # retreiving price 
  try: 
    price = soup.find( "span", attrs={'id': 'priceblock_ourprice'}).string.strip().replace(',', '') 
        # we are omitting unnecessary spaces 
        # and commas form our string 
  except AttributeError: 
    price = "NA"

  print("Products price = ", price) 
  
  # saving the price in the file
  #outFile.write(f"{price},") 


  # print availiblility status 
  try: 
    available = soup.find("div", attrs={'id': 'availability'}) 
    available = available.find("span").string.strip().replace(',', '') 
  
  except AttributeError: 
    available = "NA"

  print("Availability = ", available) 
  
  # saving the availibility and closing the line 
  #outFile.write(f"{available},\n") 

  # write the data to the file
  dataLine = [f"{today}," , f"{title_string}," , f"{price}," , f"{available},\n"]
  outFile.writelines(dataLine)
  
  # closing the file 
  outFile.close()



if __name__ == '__main__': 
  # openning our url file to access URLs 
    listfile = open(drivePath+"amazon_to_track.txt", "r") 
  
    # iterating over the urls 
    for links in listfile.readlines(): 
      main(links)

    listfile.close()

      